## Data preprocessing

### drop unnescessary columns

In [42]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.init as init

import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plts
import seaborn as sns
%matplotlib inline

data = pd.read_csv('/Users/ponynie/Developer/Python_Code/IntroDMLabChula/Final_Project/children anemia.csv')
data.dropna(inplace=True)

data.drop(['Anemia level.1', 'Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)', 'When child put to breast', 'Current marital status'], axis=1, inplace=True)
data['Age in 5-year groups'] = data['Age in 5-year groups'].apply(lambda x: sum(map(int,x.split('-')))/2)
data = data[data['Had fever in last two weeks'].isin(['Yes', 'No'])]
data = data[data['Taking iron pills, sprinkles or syrup'].isin(['Yes', 'No'])]

data.head(10)


,Age in 5-year groups,Type of place of residence,Highest educational level,Wealth index combined,Births in last five years,Age of respondent at 1st birth,Anemia level,Have mosquito bed net for sleeping (from household questionnaire),Smokes cigarettes,Currently residing with husband/partner,Had fever in last two weeks,Hemoglobin level adjusted for altitude (g/dl - 1 decimal),"Taking iron pills, sprinkles or syrup"
3,27.0,Urban,Secondary,Richest,1,25,Moderate,Yes,No,Living with her,No,114.0,No
6,37.0,Urban,Secondary,Richest,2,32,Not anemic,Yes,No,Living with her,No,102.0,Yes
9,22.0,Urban,Secondary,Richest,1,19,Moderate,Yes,No,Living with her,No,113.0,Yes
12,27.0,Urban,Higher,Richest,1,24,Mild,Yes,No,Living with her,No,109.0,No
13,22.0,Urban,Higher,Richest,2,19,Mild,Yes,No,Living with her,No,96.0,Yes
18,27.0,Urban,Secondary,Richest,2,22,Moderate,Yes,No,Living with her,No,96.0,Yes
21,27.0,Urban,Secondary,Richer,1,23,Mild,Yes,No,Living with her,No,96.0,No
22,27.0,Urban,Secondary,Richer,2,20,Mild,Yes,No,Living with her,No,115.0,Yes
25,27.0,Urban,Secondary,Richer,2,20,Not anemic,Yes,No,Staying elsewhere,No,110.0,No
36,32.0,Urban,No education,Richest,3,25,Not anemic,Yes,No,Living with her,No,138.0,No


### Maps categorical to codes

In [43]:
categorical_columns = ['Type of place of residence', 'Highest educational level', 'Wealth index combined', 'Have mosquito bed net for sleeping (from household questionnaire)', 'Smokes cigarettes', 'Currently residing with husband/partner', 'Had fever in last two weeks', 'Taking iron pills, sprinkles or syrup']
numerical_columns = ['Age in 5-year groups', 'Births in last five years', 'Age of respondent at 1st birth', 'Hemoglobin level adjusted for altitude (g/dl - 1 decimal)']
label_columns = ['Anemia level']

for category in categorical_columns:
    data[category] = data[category].astype('category')
data['Anemia level'] = data['Anemia level'].astype('category')

for categorical in categorical_columns:
    print(data[categorical].cat.categories, categorical)
print(data['Anemia level'].cat.categories, "Label")

Index(['Rural', 'Urban'], dtype='object') Type of place of residence
Index(['Higher', 'No education', 'Primary', 'Secondary'], dtype='object') Highest educational level
Index(['Middle', 'Poorer', 'Poorest', 'Richer', 'Richest'], dtype='object') Wealth index combined
Index(['No', 'Yes'], dtype='object') Have mosquito bed net for sleeping (from household questionnaire)
Index(['No', 'Yes'], dtype='object') Smokes cigarettes
Index(['Living with her', 'Staying elsewhere'], dtype='object') Currently residing with husband/partner
Index(['No', 'Yes'], dtype='object') Had fever in last two weeks
Index(['No', 'Yes'], dtype='object') Taking iron pills, sprinkles or syrup
Index(['Mild', 'Moderate', 'Not anemic', 'Severe'], dtype='object') Label


### Create nparray of categorical matrix

In [44]:
categorical_np = [data[i].cat.codes.values for i in categorical_columns]
categorical_data = np.stack(categorical_np, 1)
categorical_data[:10]

array([[1, 3, 4, 1, 0, 0, 0, 0],
       [1, 3, 4, 1, 0, 0, 0, 1],
       [1, 3, 4, 1, 0, 0, 0, 1],
       [1, 0, 4, 1, 0, 0, 0, 0],
       [1, 0, 4, 1, 0, 0, 0, 1],
       [1, 3, 4, 1, 0, 0, 0, 1],
       [1, 3, 3, 1, 0, 0, 0, 0],
       [1, 3, 3, 1, 0, 0, 0, 1],
       [1, 3, 3, 1, 0, 1, 0, 0],
       [1, 1, 4, 1, 0, 0, 0, 0]], dtype=int8)

### Convert to Categorical Tensor

In [45]:
categorical_data = torch.tensor(categorical_data, dtype=torch.int64)
categorical_data[:10]

tensor([[1, 3, 4, 1, 0, 0, 0, 0],
        [1, 3, 4, 1, 0, 0, 0, 1],
        [1, 3, 4, 1, 0, 0, 0, 1],
        [1, 0, 4, 1, 0, 0, 0, 0],
        [1, 0, 4, 1, 0, 0, 0, 1],
        [1, 3, 4, 1, 0, 0, 0, 1],
        [1, 3, 3, 1, 0, 0, 0, 0],
        [1, 3, 3, 1, 0, 0, 0, 1],
        [1, 3, 3, 1, 0, 1, 0, 0],
        [1, 1, 4, 1, 0, 0, 0, 0]])

### Create nparray of numerical matrix and convert to Tensor

In [46]:
numerical_data = np.stack([data[i].values for i in numerical_columns], 1)
numerical_data = torch.tensor(numerical_data, dtype=torch.float)
numerical_data[:10]

tensor([[ 27.,   1.,  25., 114.],
        [ 37.,   2.,  32., 102.],
        [ 22.,   1.,  19., 113.],
        [ 27.,   1.,  24., 109.],
        [ 22.,   2.,  19.,  96.],
        [ 27.,   2.,  22.,  96.],
        [ 27.,   1.,  23.,  96.],
        [ 27.,   2.,  20., 115.],
        [ 27.,   2.,  20., 110.],
        [ 32.,   3.,  25., 138.]])

### Create label's Tensor

In [47]:
outputs = torch.tensor(data['Anemia level'].cat.codes.values).flatten()
outputs[200:250]

tensor([2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 2,
        3, 2, 0, 2, 1, 2, 2, 1, 2, 0, 2, 0, 0, 2, 1, 1, 2, 2, 1, 2, 2, 0, 1, 2,
        0, 0], dtype=torch.int8)

### Check correctness of dimension

In [48]:
categorical_data.shape, numerical_data.shape, outputs.shape

(torch.Size([5855, 8]), torch.Size([5855, 4]), torch.Size([5855]))

## Create Model

### Split the data to train and test set

In [49]:
total_records = data.shape[0]
test_records = int(total_records * .2) # 20% of the data for testing
train_records = total_records - test_records # 80% of the data for training

categorical_train_data = categorical_data[:train_records]
categorical_test_data = categorical_data[train_records:]
numerical_train_data = numerical_data[:train_records]
numerical_test_data = numerical_data[train_records:]
train_outputs = outputs[:train_records]
test_outputs = outputs[train_records:]

print(categorical_train_data.shape, categorical_test_data.shape)
print(numerical_train_data.shape, numerical_test_data.shape)
print(train_outputs.shape, test_outputs.shape)



torch.Size([4684, 8]) torch.Size([1171, 8])
torch.Size([4684, 4]) torch.Size([1171, 4])
torch.Size([4684]) torch.Size([1171])


In [50]:
class NeuralNetwork(nn.Module):
    def __init__(self, numerical_features_size, categorical_features_size, hidden_size, output_size):
        super().__init__()
        all_features_size = numerical_features_size + categorical_features_size
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(all_features_size, hidden_size),
            nn.BatchNorm1d(hidden_size),  
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),  
            nn.Sigmoid(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [51]:
train_data = torch.cat((categorical_train_data, numerical_train_data), dim=1).requires_grad_(True)
test_data = torch.cat((categorical_test_data, numerical_test_data), dim=1).requires_grad_(True)
print(train_data)


tensor([[  1.,   3.,   4.,  ...,   1.,  25., 114.],
        [  1.,   3.,   4.,  ...,   2.,  32., 102.],
        [  1.,   3.,   4.,  ...,   1.,  19., 113.],
        ...,
        [  0.,   0.,   4.,  ...,   2.,  27., 103.],
        [  0.,   3.,   4.,  ...,   1.,  22.,  71.],
        [  0.,   3.,   0.,  ...,   2.,  19., 107.]], requires_grad=True)


In [52]:
train_data = train_data.float()
test_data = test_data.float()
train_outputs = train_outputs.long()
test_outputs = test_outputs.long()
print(train_outputs)

tensor([1, 2, 1,  ..., 1, 2, 2])


In [53]:
numerical_features_size = numerical_data.shape[1]
categorical_features_size = categorical_data.shape[1]
hidden_size = 40
num_epochs = 20
learning_rate = 0.05
output_size = 4
batch_size = 100

model = NeuralNetwork(numerical_features_size, categorical_features_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [54]:
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
model.to(device)
train_data = train_data.to(device)
test_data = test_data.to(device)
train_outputs = train_outputs.to(device)
test_outputs = test_outputs.to(device)

In [55]:
train_dataset = TensorDataset(train_data, train_outputs)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [56]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 10 == 9:  # Print every 100 batches
            print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}")
print("Training complete.")


Epoch 1, Batch 10, Loss: 0.1240
Epoch 1, Batch 20, Loss: 0.1159


Epoch 1, Batch 30, Loss: 0.1149
Epoch 1, Batch 40, Loss: 0.1133
Epoch 1/20, Training Loss: 1.0585576295852661
Epoch 2, Batch 10, Loss: 0.1113
Epoch 2, Batch 20, Loss: 0.1139
Epoch 2, Batch 30, Loss: 0.1121
Epoch 2, Batch 40, Loss: 0.1144
Epoch 2/20, Training Loss: 1.0386468172073364
Epoch 3, Batch 10, Loss: 0.1130
Epoch 3, Batch 20, Loss: 0.1155
Epoch 3, Batch 30, Loss: 0.1135
Epoch 3, Batch 40, Loss: 0.1121
Epoch 3/20, Training Loss: 1.0729312896728516
Epoch 4, Batch 10, Loss: 0.1123
Epoch 4, Batch 20, Loss: 0.1145
Epoch 4, Batch 30, Loss: 0.1124
Epoch 4, Batch 40, Loss: 0.1119
Epoch 4/20, Training Loss: 1.2210458517074585
Epoch 5, Batch 10, Loss: 0.1123
Epoch 5, Batch 20, Loss: 0.1127
Epoch 5, Batch 30, Loss: 0.1129
Epoch 5, Batch 40, Loss: 0.1163
Epoch 5/20, Training Loss: 1.0753041505813599
Epoch 6, Batch 10, Loss: 0.1124
Epoch 6, Batch 20, Loss: 0.1134
Epoch 6, Batch 30, Loss: 0.1132
Epoch 6, Batch 40, Loss: 0.1104
Epoch 6/20, Training Loss: 1.1004762649536133
Epoch 7, Batch 10, L

In [57]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    outputs = model(test_data)
    _, predicted = torch.max(outputs.data, 1)
    print(predicted)
    print(test_outputs)
    total += test_outputs.size(0)
    correct += (predicted == test_outputs).sum().item()
    print(correct)
    print(total)

print(f"Accuracy of the model on the test data: {(100 * correct / total)}%")

tensor([1, 1, 1,  ..., 2, 1, 1])
tensor([1, 1, 2,  ..., 2, 2, 2])
465
1171
Accuracy of the model on the test data: 39.709649871904354%
